# Linear Regression with Lasso

Let $\varphi_j(x) = \cos(\pi jx)$ (modulo factors of $\pi$ :) ).
Given data points $(x_i,y_i)_{i=1}^m \subset (a,b)\times \mathbb{R}$ generated as follows: Let $X\sim \mathcal{U}[0,1]$ and let $Y = \hat f(X)+\eta$ where $\hat f$ is sparse in $(\varphi_j)_{j}$, for example $\hat f(x) = \cos(\pi x ) + \cos(5 \pi x)$ we want to estimate $\hat f$. In other words, we want to solve the statistical learning problem associated to $(X,Y)$. We will consider the hypothesis classes $\mathcal{H}_N:=\mathrm{span}\{\varphi_j:\ j = 1,\dots , N\}$.

In empirical risk minimization (ERM) we are trying to fit a function $f:(a,b)\to \mathbb{R}$, $f(x)=\sum_{j=1}^N c_j \varphi_j(x)$ such that $\sum_{i=1}^m(f(x_i)- y_i)^2$ is minimized over $\mathcal{H}_N$. As is predicted by statistical learning theory, the generalization error will grow as the complexity of $\mathcal{H}_N$ grows. In other words, the number of necessary samples has to grow with $N$. 

The aim of this notebook is to show that one can do much better if one uses an algorithm different from ERM. This algorithm is called LASSO. Similar to ERM in LASSO we solve an optimization problem but this time we minimize the *$\ell^1$-regularized* least squares error $\sum_{i=1}^m(\sum_{j=1}^N c_j \varphi_j(x_i)- y_i)^2 + \alpha \sum_{j=1}^N|c_j|$ for some regulatization parameter $\alpha$. We will see that this algorithm significantly outperforms what we might expect from statistical learning theory. A theoretical explanation for this behaviour is provided by the field of Compressive Sensing.


In the last part we solve another optimization problem; this time we minimize the *$\ell^2$-regularized* least squares error $\sum_{i=1}^m(\sum_{j=1}^N c_j \varphi_j(x_i)- y_i)^2 + \alpha \sum_{j=1}^N|c_j|^2$ for some regulatization parameter $\alpha$. This algorithm is sometimes called *Elastic Net Regularization* We will see that this algorithm again behaves a bit more similarly to what would be expected from statistical learning. It is also not hard to see that SGD  would converge to an $\ell^2$-regularized solution.

Lets make some imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model

Lets write a function that generates our data:

In [ ]:
def datagen(m=100,a=0,b=20,noiselevel = 1, noisetype = "normal", 
            regression_function = lambda x : np.cos(5*np.pi*x/20) + np.cos(x*np.pi/20)):
    t = np.random.uniform(a,b,m) 
    t = np.sort(t)              
    sig = regression_function(t)      
    if noisetype == "normal":
        noise = np.random.normal(np.zeros(np.shape(sig)))
    if noisetype == "cauchy":
        noise = np.random.standard_cauchy(np.shape(sig))
    signoise = sig+noiselevel*noise 
    signoise.reshape(m,1) 
    return t, signoise

Lets write functions that solve the ERM and LASSO problem

In [ ]:
# least squares regression code
def LeastSquares(Basisfunct , X , Y ):
    A = Basisfunct(X)
    m = np.shape(A)[0]
    G = np.zeros((m,m))
    b = np.zeros(m)
    #generate Gram matrix
    for i in range(m):
        b[i]=np.sum(np.multiply(A[i],Y))
        for j in range(m):
            G[i,j]=np.sum(np.multiply(A[i],A[j]))
    #find optimal parameters
    params = np.linalg.solve(G,b)
    #print condition number of the linear system
    cond = np.linalg.cond(G)
    #print("condition number = {}".format(cond))
    return  params 

def Lasso(Basisfunct, X , Y , alpha = 0.1):
    A = Basisfunct(X)
    m = np.shape(A)[0]
    G = np.zeros((m,m))
    b = np.zeros(m)
    #generate Gram matrix
    for i in range(m):
        b[i]=np.sum(np.multiply(A[i],Y))
        for j in range(m):
            G[i,j]=np.sum(np.multiply(A[i],A[j]))
    #find optimal parameters
    #clf = linear_model.Lasso(alpha=alpha)
    clf = linear_model.ElasticNet(alpha=alpha , l1_ratio = 1 , max_iter = 1000)
    clf.fit(G,b)
    params = clf.coef_
    #print condition number of the linear system
    cond = np.linalg.cond(G)
    #print("condition number = {}".format(cond))
    return  params 

def L2Regularization(Basisfunct, X , Y , alpha = 1.0):
    A = Basisfunct(X)
    m = np.shape(A)[0]
    G = np.zeros((m,m))
    b = np.zeros(m)
    #generate Gram matrix
    for i in range(m):
        b[i]=np.sum(np.multiply(A[i],Y))
        for j in range(m):
            G[i,j]=np.sum(np.multiply(A[i],A[j]))
    #find optimal parameters
    clf = linear_model.ElasticNet(alpha=alpha , l1_ratio = 0 , max_iter = 5000)
    clf.fit(G,b)
    params = clf.coef_
    #print condition number of the linear system
    cond = np.linalg.cond(G)
    #print("condition number = {}".format(cond))
    return  params 

def Evaluate(Basisfunct , X , params):
    #evaluate fit on validation set
    fit = np.zeros(np.shape(X))
    C = Basisfunct(X)
    for i in range(len(params)):
        fit = fit + params[i]*C[i]
    return fit

First set parameters

In [ ]:
ntrain = 30
nval = 1000
noise = 0.4
Phi = lambda x: [np.cos(np.pi*i*x/20) for i in range(100)]
regcoeffs = np.zeros(100)
#regcoeffs[1]=1
#regcoeffs[6]=1
for i in range(1,10):
    regcoeffs[-i]=i**(-1)
#regression_function = lambda x : np.cos(5*np.pi*x/20) + np.cos(x*np.pi/20)
regression_function = lambda x : Evaluate(Phi , x , regcoeffs)
X,Y = datagen(ntrain, noiselevel = noise, regression_function = regression_function)
Xval , Yval = datagen(nval, noiselevel = noise , regression_function = regression_function)
maxdeg = 200

Our regression function is very oscillatory!

In [ ]:
t=np.linspace(0,20,1000)
plt.plot(X,Y,'ko',t,regression_function(t),'r')
plt.show()

Lets look at the performance of LASSO

In [ ]:
trainerr = np.zeros([maxdeg-1])
valerr = np.zeros([maxdeg-1])
for degree in range(1,maxdeg):
    #print("degree =  {} ".format(degree))
    Basisfunct = lambda x: [np.cos(np.pi*i*x/20) for i in range(degree)]
    params = Lasso( Basisfunct, X, Y, alpha = 5)
    trainerr[degree-1] = 1/np.sqrt(len(X))*np.linalg.norm(Y - Evaluate(Basisfunct, X , params))
    valerr[degree-1]=1/np.sqrt(len(Xval))*np.linalg.norm(Yval - Evaluate(Basisfunct, Xval , params))
    t=np.linspace(0,20,1000)
    plt.plot(X,Y,'ko',t,regression_function(t),'r',t,Evaluate(Basisfunct,t,params),'b')
    plt.title("degree = {}, samples = {}".format(degree,ntrain))
    plt.show()

In [ ]:
plt.plot(trainerr, label = "Training Error")
#plt.plot(valerr, label = "Validation Error")
plt.xlabel("degree")
plt.ylabel("error")
plt.legend()
plt.show()

In [ ]:
#plt.plot(trainerr, label = "Training Error")
plt.plot(valerr, label = "Validation Error")
plt.xlabel("degree")
plt.ylabel("error")
plt.legend()
plt.show()

In [ ]:
plt.plot(trainerr, label = "Training Error")
plt.plot(valerr, label = "Validation Error")
plt.xlabel("degree")
plt.ylabel("error")
plt.title("Training and Validation Error for LASSO")
plt.legend()
plt.show()

Lets look at the performance of ERM

In [ ]:
trainerr = np.zeros([maxdeg-1])
valerr = np.zeros([maxdeg-1])
for degree in range(1,maxdeg):
    #print("degree =  {} ".format(degree))
    Basisfunct = lambda x: [np.cos(np.pi*i*x/20) for i in range(degree)]
    params = LeastSquares( Basisfunct, X, Y)
    trainerr[degree-1] = 1/np.sqrt(len(X))*np.linalg.norm(Y - Evaluate(Basisfunct, X , params))
    valerr[degree-1]=1/np.sqrt(len(Xval))*np.linalg.norm(Yval - Evaluate(Basisfunct, Xval , params))
    t=np.linspace(0,20,1000)
    plt.plot(X,Y,'ko',t,regression_function(t),'r',t,Evaluate(Basisfunct,t,params),'b')
    plt.title("degree = {}, samples = {}".format(degree,ntrain))
    plt.show()

In [ ]:
plt.plot(trainerr, label = "Training Error")
#plt.plot(valerr, label = "Validation Error")
plt.xlabel("degree")
plt.ylabel("error")
plt.legend()
plt.show()

In [ ]:
#plt.plot(trainerr, label = "Training Error")
plt.plot(valerr, label = "Validation Error")
plt.xlabel("degree")
plt.ylabel("error")
plt.legend()
plt.show()

In [ ]:
plt.plot(trainerr, label = "Training Error")
plt.plot(valerr, label = "Validation Error")
plt.xlabel("degree")
plt.ylabel("error")
plt.title("Training and Validation Error for Emprirical Risk Minimization")
plt.legend()
plt.show()

In [ ]:
valerr[-10:-1]

Finally lets try Elastic Net Regularization

In [ ]:
trainerr = np.zeros([maxdeg-1])
valerr = np.zeros([maxdeg-1])
for degree in range(1,maxdeg):
    #print("degree =  {} ".format(degree))
    Basisfunct = lambda x: [np.cos(np.pi*i*x/20) for i in range(degree)]
    params = L2Regularization( Basisfunct, X, Y, alpha = 0.1)
    trainerr[degree-1] = 1/np.sqrt(len(X))*np.linalg.norm(Y - Evaluate(Basisfunct, X , params))
    valerr[degree-1]=1/np.sqrt(len(Xval))*np.linalg.norm(Yval - Evaluate(Basisfunct, Xval , params))
    t=np.linspace(0,20,1000)
    plt.plot(X,Y,'ko',t,regression_function(t),'r',t,Evaluate(Basisfunct,t,params),'b')
    plt.title("degree = {}, samples = {}".format(degree,ntrain))
    plt.show()

In [ ]:
plt.plot(trainerr, label = "Training Error")
#plt.plot(valerr, label = "Validation Error")
plt.xlabel("degree")
plt.ylabel("error")
plt.legend()
plt.show()

In [ ]:
#plt.plot(trainerr, label = "Training Error")
plt.plot(valerr, label = "Validation Error")
plt.xlabel("degree")
plt.ylabel("error")
plt.legend()
plt.show()

In [ ]:
plt.plot(trainerr, label = "Training Error")
plt.plot(valerr, label = "Validation Error")
plt.xlabel("degree")
plt.ylabel("error")
plt.title("Training and Validation Error for Elastic Net Regularization")
plt.legend()
plt.show()

We get a double descent curve!